# Load Individual Household Electric Power Consumption
https://archive.ics.uci.edu/dataset/235/individual+household+electric+power+consumption

## Readme
- https://medium.com/@ottaviocalzone/an-intuitive-explanation-of-lstm-a035eb6ab42c
- https://www.youtube.com/watch?v=b61DPVFX03I

In [1]:
import os
import torch
from support import EnergyConsumptionDataset, EnergyConsumptionModule, build_loaders, demo_model_shapes, \
    plot_convergence, train_and_evaluate, plot_predictions_grid, plot_mse_horizon

In [ ]:
data_folder = "local_data/cache"
if not os.path.exists(data_folder):
    os.makedirs(data_folder, exist_ok=True)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [4]:
all_data = EnergyConsumptionDataset(history_samples=24 * 14, horizon_samples=24 * 3, scale=True, train_fraction=0.9)
print("data length: ", len(all_data))
print("number of channels: ", all_data.number_of_channels)

data length:  2048873
number of channels:  7


# Explore dataset

In [5]:
all_data.data_features.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [6]:
train_dataset, test_dataset, train_loader, test_loader = build_loaders(all_data, train_fraction=0.9, batch_size_train=200, batch_size_eval=2000)

In [7]:
model = EnergyConsumptionModule(input_dim=all_data.number_of_channels, output_dim=all_data.horizon_samples)

In [8]:
demo_model_shapes(model, train_loader)

x: (200, 7, 336), model(x): (200, 72), y: (200, 7, 72)


In [9]:
checkpoint_file_name = os.path.join(data_folder, "ER Admission - CNN-LSTM Fusion.checkpoint.pt")
model, progress_log, test_accuracy, test_loader = train_and_evaluate(model, train_dataset, test_dataset, device=device, n_epochs=10, batch_size=200, lr=1e-3, checkpoint_file_name=checkpoint_file_name)
plot_convergence(progress_log, test_accuracy)
plot_predictions_grid(model, test_loader, device=device, n=6)
plot_mse_horizon(model, test_loader, all_data.horizon_samples, device=device)

d:\workspace\venv\lib\site-packages\torch\nn\modules\loss.py:634: UserWarning: Using a target size (torch.Size([200, 7, 72])) that is different to the input size (torch.Size([200, 72])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (200) must match the size of tensor b (7) at non-singleton dimension 1